# 在Sagemaker中运行自定义Tensorflow的模型

- Amazon SageMaker 是一项完全托管的服务，涵盖了 ML 的整个工作流，可以标记和准备数据、选择算法、训练模型、调整和优化模型以便部署、预测和执行操作。
- 本次实验将介绍如何使用Amazon SageMaker 的自定义脚本 （Bring Your Own Script，简称BYOS）方式来运行自定义Tensorflow的训练任务，并且进行部署调用。

## 解决方案概览

本次实验我们将使用Amazon SageMaker执行以下操作：
1. 检查自定义Tensorflow训练脚本 [ncf.py](./ncf.py) ；
2. 基于 [Tensorflow Estimator API](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html) 使用 BYOS方式进行模型训练训练模型；
3. 使用Amazon SageMaker Hosting Services部署并托管经过训练的模型作为终端节点；
4. 通过调用模型端点执行批量推理任务；


### 实验材料

1. wget
2. jupyter notebook Lab 手动上传


### Sample Train Data

选择以电影点评网站对大量电影的评分作为训练数据集；提前做好数据处理去除多余的特征和字段并且对数据集进行训练集和测试集的拆分；本次实验使用预加载数据的方式读入训练数据集，大数据场景下建议使用TFRecord方式

In [9]:
# get data
!wget https://sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com/train.npy
!wget https://sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com/test.npy

--2020-11-11 15:39:19--  https://sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com/train.npy
Resolving sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com (sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com)... 52.219.41.23
Connecting to sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com (sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com)|52.219.41.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13642112 (13M) [binary/octet-stream]
Saving to: ‘train.npy’

train.npy           100%[===================>]  13.01M  57.1MB/s    in 0.2s    

2020-11-11 15:39:19 (57.1 MB/s) - ‘train.npy’ saved [13642112/13642112]

--2020-11-11 15:39:19--  https://sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com/test.npy
Resolving sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.com (sagemaker-ap-southeast-1-249517808360.s3-ap-southeast-1.amazonaws.co

### 数据预处理

- 下载好数据后，可以在Notebook上通过python的numpy, pandas等库进行数据的读取，浏览和预处理；
- 考虑到演示的目的和运算的效率，训练模型时使用部分数据，总量包含大约56万条，用户数600+，电影数9000+
- 预测方式为二分类（即结果为判断“喜爱”或是“不喜爱”）

In [21]:
import numpy as np

train = np.load("train.npy")

print(train)
print(train.shape)

[[    1   151     1]
 [    1  1090     1]
 [    1  1224     1]
 ...
 [  532  3713     0]
 [  532 26504     0]
 [  532  2420     0]]
(568416, 3)


In [11]:
n_user = 610
n_item = 9724

### 环境准备
- 引入SageMaker的SDK，并建立SageMaker的session；
- 定义对应的S3 bucket 和 role

In [22]:
# import requirements
import sagemaker
import os
import json
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# get current SageMaker session's execution role and default bucket name
sagemaker_session = sagemaker.Session()

role = get_execution_role()
print("execution role ARN:", role)

bucket_name = sagemaker_session.default_bucket()
print("default bucket name:", bucket_name)

execution role ARN: arn:aws:iam::249517808360:role/service-role/AmazonSageMaker-ExecutionRole-20191211T161371
default bucket name: sagemaker-ap-southeast-1-249517808360


- 通过Boto3接口将数据上传到S3并定义训练数据路径

In [12]:
import boto3

region = boto3.session.Session().region_name

s3_client = boto3.client('s3', region_name=region)

s3_client.upload_file('train.npy', bucket_name, 'recommendation/data/train.npy')

training_data_uri = 's3://{}/recommendation/data/train.npy'.format(bucket_name)
print(training_data_uri)

s3://sagemaker-ap-southeast-1-249517808360/recommendation/data/train.npy


### 自定义基于Tensorflow协同过滤算法
- 数据读取
- 模型定义
- 超参数定义

In [6]:
# inspect the training script using `pygmentize` magic
!pygmentize 'ncf.py'

"""

 Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
 SPDX-License-Identifier: MIT-0
 
 Permission is hereby granted, free of charge, to any person obtaining a copy of this
 software and associated documentation files (the "Software"), to deal in the Software
 without restriction, including without limitation the rights to use, copy, modify,
 merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
 permit persons to whom the Software is furnished to do so.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
 INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
 PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
 OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
 SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

"""


import tensorflow

In [14]:
training_parameters = {
    'epochs': 10,
    'batch_size': 256, 
    'n_user': n_user, 
    'n_item': n_item
}

### 环境参数定义
包括之前定义好的训练脚本、角色、输出位置和session、还有训练使用的EC2实例

    entry_point, 需要运行的脚本
    role=role,训练和推理过程用用于获取AWS资源的 IAM Role
    train_instance_count,设置用于训练实例数量，可以设置多台做分布式训练用于训练的实例数量
    train_instance_type,设置训练实例类型
    framework_version='2.1.0',TF版本号
    py_version='py3',python 版本
    distributions={'parameter_server': {'enabled': True}},基于CPU做分布式训练
    hyperparameters=training_parameters,训练超参数指定
    train_use_spot_instances,训练过程是否使用Spot资源

In [15]:
# initiate the training job using Tensorflow estimator
ncf_estimator = TensorFlow(
    entry_point='ncf.py',
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    framework_version='2.1.0',
    py_version='py3',
    distributions={'parameter_server': {'enabled': True}},
    hyperparameters=training_parameters,
    train_use_spot_instances=True, train_max_run=3600, train_max_wait=7200
)

Parameter distribution will be renamed to {'parameter_server': {'enabled': True}} in SageMaker Python SDK v2.


In [ ]:
#基于GPU做分布式训练
# tensorflow支持的分布式：parameter server & horovod & native distrustrategy
#horovod sample code
'''
distributions={'mpi':{'enabled':'True','processes_per_host':4}}
'''
#基于CPU做分布式训练
#sample code
'''
distributions={'parameter_server':'True'}
'''

### 启动训练任务

1. 为模型提供训练集和测试集在S3中的位置

2. 调用fit api 开始训练


In [16]:
# kick off the training job
ncf_estimator.fit(training_data_uri)

'''
mnist_estimator.fit(inputs={'train':'s3://sagemaker-ap-southeast-1-937315424903/data/MNIST-NPY'},
                    job_name="Run-MNIST2-Training-Script",
                    wait=True,            #是否等待训练执行。False为异步
                    logs=True,
                    run_tensorboard_locally=True)
'''

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-11-11 15:41:01 Starting - Starting the training job...
2020-11-11 15:41:03 Starting - Launching requested ML instances......
2020-11-11 15:42:30 Starting - Preparing the instances for training.........
2020-11-11 15:43:58 Downloading - Downloading input data
2020-11-11 15:43:58 Training - Downloading the training image..2020-11-11 15:44:15,207 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-11-11 15:44:15,213 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-11 15:44:15,535 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-11 15:44:15,550 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-11 15:44:15,565 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-11 15:44:15,576 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {
        "sagemaker_par

### 训练任务可视化
在Amazon SageMaker控制台看到训练任务，点进详情可在cloudwatch看到训练的日志输出，以及监控机器的GPU、CPU、内存等的使用率等情况

也可考虑一下应用SageMaker的超参调优（Hyperparameters Tuning）来尝试其他的超参设置是否可以获得更好的二分准确度


## Deploy the Endpoint

### 一键部署推理实例
完成训练后，我们可以轻松的将上面的模型部署成一个实时可在生产环境中调用的端口
- 定义推理集群数量和实例类型

In [17]:
# once the model is trained, we can deploy the model using Amazon SageMaker Hosting Services
# Here we deploy the model using one ml.c5.xlarge instance as a tensorflow-serving endpoint
# This enables us to invoke the endpoint like how we use Tensorflow serving
# Read more about Tensorflow serving using the link below
# https://www.tensorflow.org/tfx/tutorials/serving/rest_simple

endpoint_name = 'neural-collaborative-filtering-model-demo'

predictor = ncf_estimator.deploy(initial_instance_count=1, 
                                 instance_type='ml.m5.large', 
                                 endpoint_type='tensorflow-serving',
                                 endpoint_name=endpoint_name)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

## Invoke

相应的endpoint成功创建后，可以通过API的方式调用该模型实现推荐

In [11]:
# To use the endpoint in another notebook, we can initiate a predictor object as follows
from sagemaker.tensorflow import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name)

### 读取测试数据

In [24]:
# Define a function to read testing data
def _load_testing_data(filename):
    """ load testing data """
    df_test = np.load(filename)
    user_test, item_test, y_test = np.split(np.transpose(df_test).flatten(), 3)
    return user_test, item_test, y_test
# read testing data from local
user_test, item_test, test_labels = _load_testing_data('test.npy')

# one-hot encode the testing data for model input
with tf.Session() as tf_sess:
    test_user_data = tf_sess.run(tf.one_hot(user_test, depth=n_user)).tolist()
    test_item_data = tf_sess.run(tf.one_hot(item_test, depth=n_item)).tolist()

### 通过调用sdk的.predict()直接调用定义的predictor 对象

In [14]:
# make batch prediction
batch_size = 100
y_pred = []
for idx in range(0, len(test_user_data), batch_size):
    # reformat test samples into tensorflow serving acceptable format
    input_vals = {
     "instances": [
         {'input_1': u, 'input_2': i} 
         for (u, i) in zip(test_user_data[idx:idx+batch_size], test_item_data[idx:idx+batch_size])
    ]}
 
    # invoke model endpoint to make inference
    pred = predictor.predict(input_vals)
    
    # store predictions
    y_pred.extend([i[0] for i in pred['predictions']])

- 通过自定义置信度阈值来控制输出结果

In [15]:
# let's see some prediction examples, assuming the threshold 
# --- prediction probability view ---
print('This is what the prediction output looks like')
print(y_pred[:5], end='\n\n\n')

# --- user item pair prediction view, with threshold of 0.5 applied ---
pred_df = pd.DataFrame([
    user_test,
    item_test,
    (np.array(y_pred) >= 0.5).astype(int)],
).T

pred_df.columns = ['userId', 'movieId', 'prediction']

print('We can convert the output to user-item pair as shown below')
print(pred_df.head(), end='\n\n\n')

# --- aggregated prediction view, by user ---
print('Lastly, we can roll up the prediction list by user and view it that way')
print(pred_df.query('prediction == 1').groupby('userId').movieId.apply(list).head().to_frame(), end='\n\n\n')

This is what the prediction output looks like
[0.0870951936, 0.809446931, 0.361209929, 0.666578412, 0.0646538213]


We can convert the output to user-item pair as shown below
   userId  movieId  prediction
0       1     2492           0
1       1     2012           1
2       1     2478           0
3       1      553           1
4       1     1445           0


Lastly, we can roll up the prediction list by user and view it that way
                         movieId
userId                          
1         [2012, 553, 780, 3448]
2                         [1704]
3        [2424, 527, 3949, 1302]
4                   [4246, 4896]
5       [300, 474, 266, 36, 253]




## Delete Endpoint

实验完毕及时清理资源，删除上一步部署好的推理节点endpoint

In [16]:
# delete endpoint at the end of the demo
predictor.delete_endpoint(delete_endpoint_config=True)